In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
data_dir = '../data/'
df_demo = pd.DataFrame(pd.read_excel(data_dir + 'CHEXPERT_DEMO.xlsx', engine='openpyxl'))
# DF demo contains extra information about race and ethnicity
df_demo = df_demo.rename(columns={'PRIMARY_RACE': 'race'})
df_demo = df_demo.rename(columns={'PATIENT': 'patient_id'})
df_demo = df_demo.rename(columns={'GENDER': 'sex'})
df_demo = df_demo.rename(columns={'AGE_AT_CXR': 'age'})
df_demo = df_demo.rename(columns={'ETHNICITY': 'ethnicity'})
df_demo = df_demo.drop(['sex', 'age'], axis=1)
print("df demo data")
print(df_demo.shape)
df_demo.head()


df demo data
(65401, 3)


,patient_id,race,ethnicity
0,patient24428,White,Non-Hispanic/Non-Latino
1,patient48289,Other,Hispanic/Latino
2,patient33856,White,Non-Hispanic/Non-Latino
3,patient41673,Unknown,Unknown
4,patient48493,White,Non-Hispanic/Non-Latino


In [3]:
df_data_split = pd.read_csv(data_dir + 'chexpert_split_2021_08_20.csv').set_index('index')
print(df_data_split.shape)
df_data_split.head()

(127118, 2)


,Unnamed: 0,split
index,,
1,0,train
2,1,train
4,2,train
5,3,train
7,4,train


In [4]:
df_img_data_original_chexbert= pd.read_csv(data_dir + 'train_cheXbert.csv')
print("df_img_data_original_chexbert")
print(df_img_data_original_chexbert.shape)
df_img_data_original_chexbert.head()

df_img_data_original_chexbert
(223414, 19)


,Path,Sex,Age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,NaN,1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,NaN
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [5]:

split =  df_img_data_original_chexbert.Path.str.split("/", expand = True) # new data frame
print(split.shape)
split.head()


(223414, 5)


,0,1,2,3,4
0,CheXpert-v1.0,train,patient00001,study1,view1_frontal.jpg
1,CheXpert-v1.0,train,patient00002,study2,view1_frontal.jpg
2,CheXpert-v1.0,train,patient00002,study1,view1_frontal.jpg
3,CheXpert-v1.0,train,patient00002,study1,view2_lateral.jpg
4,CheXpert-v1.0,train,patient00003,study1,view1_frontal.jpg


In [6]:
df_img_data = df_img_data_original_chexbert
df_img_data["patient_id"] = split[2]
df_img_data = df_img_data.rename(columns={'Age': 'age'})
df_img_data = df_img_data.rename(columns={'Sex': 'sex'})
print(df_img_data.shape)
df_img_data.head()

(223414, 20)


,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,patient_id
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,patient00001
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,NaN,1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,NaN,patient00002
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,patient00002
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,patient00002
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,patient00003


In [7]:
df_cxr = df_demo.merge(df_img_data, on="patient_id")
print(df_cxr.shape)
df_cxr.head()

(222561, 22)


,patient_id,race,ethnicity,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study22/view1...,Male,59,Frontal,AP,NaN,NaN,...,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view1...,Male,61,Frontal,PA,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
2,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view2...,Male,61,Lateral,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
3,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study61/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
4,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study35/view1...,Male,60,Frontal,AP,NaN,NaN,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN


In [8]:
white = 'White'
asian = 'Asian'
black = 'Black'

In [9]:
# Only takes 
mask = (df_cxr.race.str.contains("Black", na=False))
df_cxr.loc[mask, "race"] = black

mask = (df_cxr.race.str.contains("White", na=False))
df_cxr.loc[mask, "race"] = white

mask = (df_cxr.race.str.contains("Asian", na=False))
df_cxr.loc[mask, "race"] = asian

df_cxr['race'].unique()
print(df_cxr.shape)
df_cxr.head()

(222561, 22)


,patient_id,race,ethnicity,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study22/view1...,Male,59,Frontal,AP,NaN,NaN,...,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view1...,Male,61,Frontal,PA,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
2,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view2...,Male,61,Lateral,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
3,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study61/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
4,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study35/view1...,Male,60,Frontal,AP,NaN,NaN,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN


In [10]:
df_cxr = df_cxr[df_cxr.race.isin([asian,black,white])]
print(df_cxr.shape)
df_cxr.head()

(160724, 22)


,patient_id,race,ethnicity,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study22/view1...,Male,59,Frontal,AP,NaN,NaN,...,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view1...,Male,61,Frontal,PA,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
2,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view2...,Male,61,Lateral,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
3,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study61/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
4,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study35/view1...,Male,60,Frontal,AP,NaN,NaN,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN


In [11]:

df_cxr = df_cxr[df_cxr.ethnicity.isin(["Non-Hispanic/Non-Latino","Not Hispanic"])]
print(df_cxr.shape)
df_cxr.head()

(149283, 22)


,patient_id,race,ethnicity,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study22/view1...,Male,59,Frontal,AP,NaN,NaN,...,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view1...,Male,61,Frontal,PA,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
2,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view2...,Male,61,Lateral,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
3,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study61/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
4,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study35/view1...,Male,60,Frontal,AP,NaN,NaN,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN


In [12]:
df_cxr = df_cxr[df_cxr["Frontal/Lateral"]=="Frontal"]
print(df_cxr.shape)
df_cxr.head()

(127130, 22)


,patient_id,race,ethnicity,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study22/view1...,Male,59,Frontal,AP,NaN,NaN,...,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view1...,Male,61,Frontal,PA,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
3,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study61/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
4,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study35/view1...,Male,60,Frontal,AP,NaN,NaN,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN
5,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study45/view1...,Male,61,Frontal,AP,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN


In [13]:
df_cxr['race_label'] = df_cxr['race']

df_cxr.loc[df_cxr['race_label'] == white, 'race_label'] = 0
df_cxr.loc[df_cxr['race_label'] == asian, 'race_label'] = 1
df_cxr.loc[df_cxr['race_label'] == black, 'race_label'] = 2
print(df_cxr.shape)
df_cxr.head()

(127130, 23)


,patient_id,race,ethnicity,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,...,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,race_label
0,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study22/view1...,Male,59,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0
1,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view1...,Male,61,Frontal,PA,NaN,NaN,...,0.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0
3,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study61/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,0
4,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study35/view1...,Male,60,Frontal,AP,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0
5,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study45/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,0


In [14]:
df_cxr['sex_label'] = df_cxr['sex']

df_cxr.loc[df_cxr['sex_label'] == 'Male', 'sex_label'] = 0
df_cxr.loc[df_cxr['sex_label'] == 'Female', 'sex_label'] = 1
print(df_cxr.shape)
df_cxr.head()

(127130, 24)


,patient_id,race,ethnicity,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,...,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,race_label,sex_label
0,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study22/view1...,Male,59,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0,0
1,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view1...,Male,61,Frontal,PA,NaN,NaN,...,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0,0
3,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study61/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,0,0
4,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study35/view1...,Male,60,Frontal,AP,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0,0
5,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study45/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,0,0


In [15]:
labels = [
    'No Finding',
    'Enlarged Cardiomediastinum',
    'Cardiomegaly',
    'Lung Opacity',
    'Lung Lesion',
    'Edema',
    'Consolidation',
    'Pneumonia',
    'Atelectasis',
    'Pneumothorax',
    'Pleural Effusion',
    'Pleural Other',
    'Fracture',
    'Support Devices']

In [16]:

df_cxr['disease'] = df_cxr[labels[0]]
df_cxr.loc[df_cxr[labels[0]] == 1, 'disease'] = labels[0]
df_cxr.loc[df_cxr[labels[10]] == 1, 'disease'] = labels[10]
df_cxr.loc[df_cxr['disease'].isna(), 'disease'] = 'Other'

df_cxr['disease_label'] = df_cxr['disease']
df_cxr.loc[df_cxr['disease_label'] == labels[0], 'disease_label'] = 0
df_cxr.loc[df_cxr['disease_label'] == labels[10], 'disease_label'] = 1
df_cxr.loc[df_cxr['disease_label'] == 'Other', 'disease_label'] = 2
print(df_cxr.shape)
df_cxr.head()


(127130, 26)


/tmp/ipykernel_19242/3501818218.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'No Finding' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_cxr.loc[df_cxr[labels[0]] == 1, 'disease'] = labels[0]


,patient_id,race,ethnicity,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,...,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,race_label,sex_label,disease,disease_label
0,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study22/view1...,Male,59,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,0,0,Other,2
1,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view1...,Male,61,Frontal,PA,NaN,NaN,...,NaN,-1.0,NaN,NaN,NaN,NaN,0,0,Other,2
3,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study61/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,1.0,NaN,NaN,1.0,NaN,0,0,Pleural Effusion,1
4,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study35/view1...,Male,60,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,0,0,Other,2
5,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study45/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,1.0,NaN,NaN,1.0,NaN,0,0,Pleural Effusion,1


In [17]:
from sklearn.utils import shuffle
df_cxr.insert(5, "split","none", True)
unique_sub_id = df_cxr.patient_id.unique()

train_percent, valid_percent, test_percent = 0.60, 0.10, 0.30

unique_sub_id = shuffle(unique_sub_id)
value1 = (round(len(unique_sub_id)*train_percent))
value2 = (round(len(unique_sub_id)*valid_percent))
value3 = value1 + value2
value4 = (round(len(unique_sub_id)*test_percent))
print(df_cxr.shape)
df_cxr.head()


(127130, 27)


,patient_id,race,ethnicity,Path,sex,split,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,...,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,race_label,sex_label,disease,disease_label
0,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study22/view1...,Male,none,59,Frontal,AP,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,0,0,Other,2
1,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view1...,Male,none,61,Frontal,PA,NaN,...,NaN,-1.0,NaN,NaN,NaN,NaN,0,0,Other,2
3,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study61/view1...,Male,none,61,Frontal,AP,NaN,...,NaN,1.0,NaN,NaN,1.0,NaN,0,0,Pleural Effusion,1
4,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study35/view1...,Male,none,60,Frontal,AP,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,0,0,Other,2
5,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study45/view1...,Male,none,61,Frontal,AP,NaN,...,NaN,1.0,NaN,NaN,1.0,NaN,0,0,Pleural Effusion,1


In [18]:
df_cxr = shuffle(df_cxr)

train_sub_id = unique_sub_id[:value1]
validate_sub_id = unique_sub_id[value1:value3]
test_sub_id = unique_sub_id[value3:]

In [19]:
df_cxr.loc[df_cxr.patient_id.isin(train_sub_id), "split"]="train"
df_cxr.loc[df_cxr.patient_id.isin(validate_sub_id), "split"]="validate"
df_cxr.loc[df_cxr.patient_id.isin(test_sub_id), "split"]="test"
print(df_cxr.shape)
df_cxr.head()

(127130, 27)


,patient_id,race,ethnicity,Path,sex,split,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,...,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,race_label,sex_label,disease,disease_label
20972,patient39499,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient39499/study2/view1_...,Male,train,48,Frontal,AP,NaN,...,NaN,0.0,NaN,NaN,1.0,1.0,0,0,No Finding,0
65481,patient12809,Asian,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient12809/study16/view1...,Female,train,67,Frontal,AP,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,1,1,Pleural Effusion,1
135322,patient17876,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient17876/study6/view1_...,Male,test,91,Frontal,AP,NaN,...,NaN,1.0,NaN,NaN,1.0,NaN,0,0,Pleural Effusion,1
97389,patient19890,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient19890/study1/view1_...,Female,train,69,Frontal,AP,NaN,...,0.0,0.0,NaN,NaN,1.0,NaN,0,1,Other,2
44874,patient02395,Asian,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient02395/study10/view1...,Female,train,75,Frontal,AP,NaN,...,NaN,1.0,NaN,NaN,1.0,NaN,1,1,Pleural Effusion,1


In [20]:
df_cxr.split.value_counts(normalize=True)

split
train       0.598930
test        0.297546
validate    0.103524
Name: proportion, dtype: float64

In [21]:
df_cxr.race.value_counts()

race
White    99037
Asian    18830
Black     9263
Name: count, dtype: int64

In [22]:
df_cxr.race.value_counts(normalize=True)

race
White    0.779021
Asian    0.148116
Black    0.072862
Name: proportion, dtype: float64

In [23]:
df_cxr = df_cxr.sort_values(by=['Path'])
df_cxr = df_cxr.reset_index()
df_cxr = df_cxr.drop(columns=['index'])
df_cxr = df_cxr[df_cxr["AP/PA"].isin(['AP','PA'])]
print(df_cxr.shape)


(127118, 27)


In [24]:
train_df = df_cxr[df_cxr.split=="train"]
validation_df = df_cxr[df_cxr.split=="validate"]
test_df = df_cxr[df_cxr.split=="test"]

In [25]:
#False indicates no patient_id shared between groups
import numpy as np
unique_train_id = train_df.patient_id.unique()
unique_validation_id = validation_df.patient_id.unique()
unique_test_id = test_df.patient_id.unique()
all_id = np.concatenate((unique_train_id, unique_validation_id, unique_test_id), axis=None)

def contains_duplicates(X):
    return len(np.unique(X)) != len(X)

print(contains_duplicates(all_id))
print(df_cxr.shape)
df_cxr.head()

False
(127118, 27)


,patient_id,race,ethnicity,Path,sex,split,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,...,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,race_label,sex_label,disease,disease_label
0,patient00002,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,train,83,Frontal,AP,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,0,1,Other,2
1,patient00002,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,train,87,Frontal,AP,NaN,...,NaN,-1.0,NaN,1.0,NaN,NaN,0,1,Other,2
2,patient00003,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,train,41,Frontal,AP,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,0,0,Other,2
3,patient00004,Black,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient00004/study1/view1_...,Female,test,20,Frontal,PA,0.0,...,NaN,0.0,NaN,NaN,NaN,1.0,2,1,No Finding,0
4,patient00005,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient00005/study1/view1_...,Male,train,33,Frontal,PA,1.0,...,NaN,0.0,NaN,NaN,1.0,1.0,0,0,No Finding,0
